In [ ]:
#注意事项:
#当运行本Notebook的程序后，如果要关闭Notebook，请选择菜单: File > Close and Halt 才能确实停止当前正在运行的程序，并且释放资源
#如果没有使用以上方法，只关闭此分页，程序仍在运行，未释放资源，当您打开并运行其他的Notebook，可能会发生错误

# 21	Spark ML Pipeline多元分类

In [1]:
sc.master

u'local[*]'

# 21.1	数据准备

In [2]:
global Path    
if sc.master[0:5]=="local" :
   Path="file:/home/hduser/pythonsparkexample/PythonProject/"
else:   
   Path="hdfs://master:9000/user/hduser/"
#如果要在cluster模式运行(hadoop yarn 或Spark Stand alone)，请按照书上的说明，先把文件上传到HDFS目录

In [3]:
rawData = sc.textFile(Path+"data/covtype.data")
lines = rawData.map(lambda x: x.split(","))
lines.count()

581012

In [4]:
fieldnum = len(lines.first()) 
fieldnum

55

In [5]:
from pyspark.sql.types import  StringType,StructField,StructType

In [6]:
fields = [StructField("f"+str(i), StringType(), True) 
               for i in range(fieldnum )]

In [7]:
schema = StructType(fields)

In [8]:
covtype_df = spark.createDataFrame(lines, schema)

In [9]:
from pyspark.sql.types import  StringType
fields = [StructField("f"+str(i), StringType(), True) for i in range(fieldnum )]
schema = StructType(fields)
covtype_df = spark.createDataFrame(lines, schema)

In [10]:
print covtype_df.columns

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54']


In [11]:
print covtype_df.printSchema()

root
 |-- f0: string (nullable = true)
 |-- f1: string (nullable = true)
 |-- f2: string (nullable = true)
 |-- f3: string (nullable = true)
 |-- f4: string (nullable = true)
 |-- f5: string (nullable = true)
 |-- f6: string (nullable = true)
 |-- f7: string (nullable = true)
 |-- f8: string (nullable = true)
 |-- f9: string (nullable = true)
 |-- f10: string (nullable = true)
 |-- f11: string (nullable = true)
 |-- f12: string (nullable = true)
 |-- f13: string (nullable = true)
 |-- f14: string (nullable = true)
 |-- f15: string (nullable = true)
 |-- f16: string (nullable = true)
 |-- f17: string (nullable = true)
 |-- f18: string (nullable = true)
 |-- f19: string (nullable = true)
 |-- f20: string (nullable = true)
 |-- f21: string (nullable = true)
 |-- f22: string (nullable = true)
 |-- f23: string (nullable = true)
 |-- f24: string (nullable = true)
 |-- f25: string (nullable = true)
 |-- f26: string (nullable = true)
 |-- f27: string (nullable = true)
 |-- f28: string (nullabl

In [12]:
featuresCols=covtype_df.columns[:54]
print featuresCols

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53']


In [13]:
from pyspark.sql.functions import col  
covtype_df= covtype_df.select([ col(column).cast("double").alias(column) 
                                          for column in covtype_df.columns])

In [14]:
covtype_df.printSchema()

root
 |-- f0: double (nullable = true)
 |-- f1: double (nullable = true)
 |-- f2: double (nullable = true)
 |-- f3: double (nullable = true)
 |-- f4: double (nullable = true)
 |-- f5: double (nullable = true)
 |-- f6: double (nullable = true)
 |-- f7: double (nullable = true)
 |-- f8: double (nullable = true)
 |-- f9: double (nullable = true)
 |-- f10: double (nullable = true)
 |-- f11: double (nullable = true)
 |-- f12: double (nullable = true)
 |-- f13: double (nullable = true)
 |-- f14: double (nullable = true)
 |-- f15: double (nullable = true)
 |-- f16: double (nullable = true)
 |-- f17: double (nullable = true)
 |-- f18: double (nullable = true)
 |-- f19: double (nullable = true)
 |-- f20: double (nullable = true)
 |-- f21: double (nullable = true)
 |-- f22: double (nullable = true)
 |-- f23: double (nullable = true)
 |-- f24: double (nullable = true)
 |-- f25: double (nullable = true)
 |-- f26: double (nullable = true)
 |-- f27: double (nullable = true)
 |-- f28: double (nullabl

In [15]:
covtype_df=covtype_df.withColumn("label", covtype_df["f54"] - 1).drop("f54")

In [16]:
covtype_df.show(1)

+------+----+---+-----+---+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|    f0|  f1| f2|   f3| f4|   f5|   f6|   f7|   f8|    f9|f10|f11|f12|f13|f14|f15|f16|f17|f18|f19|f20|f21|f22|f23|f24|f25|f26|f27|f28|f29|f30|f31|f32|f33|f34|f35|f36|f37|f38|f39|f40|f41|f42|f43|f44|f45|f46|f47|f48|f49|f50|f51|f52|f53|label|
+------+----+---+-----+---+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|2596.0|51.0|3.0|258.0|0.0|510.0|221.0|232.0|148.0|6279.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|  4.0|
+------+----+---+-----+---+-----+---

In [17]:
covtype_df.first()

Row(f0=2596.0, f1=51.0, f2=3.0, f3=258.0, f4=0.0, f5=510.0, f6=221.0, f7=232.0, f8=148.0, f9=6279.0, f10=1.0, f11=0.0, f12=0.0, f13=0.0, f14=0.0, f15=0.0, f16=0.0, f17=0.0, f18=0.0, f19=0.0, f20=0.0, f21=0.0, f22=0.0, f23=0.0, f24=0.0, f25=0.0, f26=0.0, f27=0.0, f28=0.0, f29=0.0, f30=0.0, f31=0.0, f32=0.0, f33=0.0, f34=0.0, f35=0.0, f36=0.0, f37=0.0, f38=0.0, f39=0.0, f40=0.0, f41=0.0, f42=1.0, f43=0.0, f44=0.0, f45=0.0, f46=0.0, f47=0.0, f48=0.0, f49=0.0, f50=0.0, f51=0.0, f52=0.0, f53=0.0, label=4.0)

In [18]:
covtype_df

DataFrame[f0: double, f1: double, f2: double, f3: double, f4: double, f5: double, f6: double, f7: double, f8: double, f9: double, f10: double, f11: double, f12: double, f13: double, f14: double, f15: double, f16: double, f17: double, f18: double, f19: double, f20: double, f21: double, f22: double, f23: double, f24: double, f25: double, f26: double, f27: double, f28: double, f29: double, f30: double, f31: double, f32: double, f33: double, f34: double, f35: double, f36: double, f37: double, f38: double, f39: double, f40: double, f41: double, f42: double, f43: double, f44: double, f45: double, f46: double, f47: double, f48: double, f49: double, f50: double, f51: double, f52: double, f53: double, label: double]

In [19]:
train_df, test_df = covtype_df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache()

DataFrame[f0: double, f1: double, f2: double, f3: double, f4: double, f5: double, f6: double, f7: double, f8: double, f9: double, f10: double, f11: double, f12: double, f13: double, f14: double, f15: double, f16: double, f17: double, f18: double, f19: double, f20: double, f21: double, f22: double, f23: double, f24: double, f25: double, f26: double, f27: double, f28: double, f29: double, f30: double, f31: double, f32: double, f33: double, f34: double, f35: double, f36: double, f37: double, f38: double, f39: double, f40: double, f41: double, f42: double, f43: double, f44: double, f45: double, f46: double, f47: double, f48: double, f49: double, f50: double, f51: double, f52: double, f53: double, label: double]

# 21.2	建立机器学习pipeline管线

In [20]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import   VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

In [21]:
vectorAssembler = VectorAssembler(inputCols=featuresCols, 
                                                               outputCol="features")
dt = DecisionTreeClassifier(labelCol="label",featuresCol= 'features'
                                             ,maxDepth =5,maxBins=20)
dt_pipeline = Pipeline(stages=[vectorAssembler ,dt])

In [22]:
dt_pipeline.getStages()

[VectorAssembler_43a7a2906cdd94576eee,
 DecisionTreeClassifier_4e15848b4a4cc2cb804d]

# 21.3	使用pipeline进行数据处理与训练

In [23]:
pipelineModel=dt_pipeline.fit(train_df)

In [24]:
pipelineModel.stages[1]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4e15848b4a4cc2cb804d) of depth 5 with 63 nodes

In [25]:
print pipelineModel.stages[1].toDebugString[:500]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4e15848b4a4cc2cb804d) of depth 5 with 63 nodes
  If (feature 0 <= 3057.0)
   If (feature 0 <= 2585.0)
    If (feature 10 <= 0.0)
     If (feature 0 <= 2407.0)
      If (feature 3 <= 0.0)
       Predict: 3.0
      Else (feature 3 > 0.0)
       Predict: 2.0
     Else (feature 0 > 2407.0)
      If (feature 17 <= 0.0)
       Predict: 2.0
      Else (feature 17 > 0.0)
       Predict: 2.0
    Else (feature 10 > 0.0)
     If (feature 22 <= 0.0


# 21.4	使用pipelineModel 进行预测

In [26]:
predicted = pipelineModel.transform(test_df)

In [27]:
print predicted .columns

['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'label', 'features', 'rawPrediction', 'probability', 'prediction']


In [28]:
predicted.select('features', 'rawPrediction', 'probability', 'label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|            features|       rawPrediction|         probability|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  5.0|       2.0|
|(54,[0,1,2,5,6,7,...|[0.0,40.0,449.0,7...|[0.0,0.0262295081...|  5.0|       3.0|
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  2.0|       2.0|
|(54,[0,1,2,5,6,7,...|[0.0,40.0,449.0,7...|[0.0,0.0262295081...|  5.0|       3.0|
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,451.0,12120....|[0.0,0.0240072394...|  5.0|       2.0|
|(54,[0,1,2,3,4,

In [29]:
predicted.select('probability', 'prediction').take(10)

[Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0262, 0.2944, 0.4846, 0.0, 0.1948, 0.0]), prediction=3.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0262, 0.2944, 0.4846, 0.0, 0.1948, 0.0]), prediction=3.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.024, 0.6452, 0.0599, 0.0, 0.2709, 0.0]), prediction=2.0)]

# 21.5	评估模型的准确率

In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
evaluator = MulticlassClassificationEvaluator(
                            labelCol="label", predictionCol="prediction", 
                            metricName="accuracy")

In [32]:
predictions=pipelineModel.transform(test_df)
accuracy = evaluator.evaluate(predictions)
accuracy 

0.7005240585294252

In [33]:
#21.6	使用TrainValidation进行训练评估找出最佳模型

In [34]:
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit

In [35]:
paramGrid = ParamGridBuilder()\
  .addGrid(dt.impurity, [ "gini","entropy"])\
  .addGrid(dt.maxDepth, [ 10,15,25])\
  .addGrid(dt.maxBins, [30,40,50])\
  .build()

In [36]:
tvs = TrainValidationSplit(estimator=dt,evaluator=evaluator,
                  estimatorParamMaps=paramGrid,trainRatio=0.8)    

In [37]:
tvs_pipeline = Pipeline(stages=[vectorAssembler , tvs])

In [38]:
tvs_pipelineModel =tvs_pipeline.fit(train_df)

In [39]:
bestModel=tvs_pipelineModel.stages[1].bestModel
print bestModel.toDebugString[:500]   

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4e15848b4a4cc2cb804d) of depth 25 with 47505 nodes
  If (feature 0 <= 2717.0)
   If (feature 0 <= 2519.0)
    If (feature 0 <= 2407.0)
     If (feature 23 <= 0.0)
      If (feature 3 <= 0.0)
       If (feature 13 <= 0.0)
        If (feature 9 <= 603.0)
         If (feature 16 <= 0.0)
          Predict: 5.0
         Else (feature 16 > 0.0)
          If (feature 5 <= 379.0)
           Predict: 5.0
          Else (feature 5 > 379.0)
      


In [40]:
predictions = tvs_pipelineModel.transform(test_df)
result=predictions.withColumnRenamed("f0", "海拔") \
                                           .withColumnRenamed("f1", "方位") \
                                           .withColumnRenamed("f2", "斜率") \
                                           .withColumnRenamed("f3", "垂直距离") \
                                           .withColumnRenamed("f4", "水平距离") \
                                           .withColumnRenamed("f5", "阴影")           
result.select("海拔","方位","斜率","垂直距离" , "水平距离","阴影","label","prediction").show(10)

+------+----+----+-----+----+-----+-----+----------+
|    海拔|  方位|  斜率| 垂直距离|水平距离|   阴影|label|prediction|
+------+----+----+-----+----+-----+-----+----------+
|1863.0|37.0|17.0|120.0|18.0| 90.0|  5.0|       5.0|
|1866.0|23.0|14.0| 85.0|16.0|108.0|  2.0|       2.0|
|1871.0|22.0|22.0| 60.0|12.0| 85.0|  5.0|       5.0|
|1871.0|36.0|19.0|134.0|26.0|120.0|  5.0|       5.0|
|1874.0|18.0|14.0|  0.0| 0.0| 90.0|  5.0|       2.0|
|1877.0|19.0|18.0| 85.0|25.0|108.0|  2.0|       2.0|
|1879.0|18.0|14.0|  0.0| 0.0|120.0|  5.0|       2.0|
|1880.0|13.0|23.0| 90.0|29.0| 67.0|  5.0|       5.0|
|1883.0|27.0|24.0|120.0|24.0|108.0|  5.0|       5.0|
|1883.0|29.0|24.0| 60.0|24.0|108.0|  5.0|       5.0|
+------+----+----+-----+----+-----+-----+----------+
only showing top 10 rows



In [41]:
accuracy = evaluator.evaluate(predictions)
accuracy 

0.9283691115086464